In [13]:
PATH = '/home/g056122/FeatExt_Data_Clustering/final_github/content_based_filtering/'

In [14]:
DATAPATH = '/home/g056122/ISG_studienarbeit/Movielens_100K'

In [15]:
''' 
Load 'validate', 'euclid' and 'valres' module available in 'Module' folder
'''
# import required libraries
import numpy as np
import pandas as pd
import datetime
from kmodes.kmodes import KModes
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from validate import evaluation
from euclid import EuclideanDistance
from valres import result
from sklearn.metrics import mean_squared_error,mean_absolute_error 
from sklearn.ensemble import RandomForestRegressor

In [16]:
# import movies dataset
dfmovies = pd.read_csv("/home/g056122/ISG_studienarbeit/Movielens_100K/u.item", sep='|', header=None, engine='python', encoding='latin-1')
dfmovies.columns = ['movieId', 'movietitle', 'releasedate', 'videoreleasedate','imdburl', 'unknown', 'action', 'adventure', 'animation','childrens', 'comedy', 'crime', 'documentary', 'drama', 'fantasy', 'film-Noir', 'horror', 'musical', 'mystery', 'romance', 'sci-fi','thriller', 'war', 'western']

In [17]:
# drop videoreleasedate and imdburl feature
dfmovies = dfmovies.drop(['videoreleasedate', 'imdburl'], axis=1)

In [18]:
# year feature transformation
dfmovies['releasedate'] = pd.to_datetime (dfmovies['releasedate'], format='%d-%b-%Y')
dfmovies['year']=dfmovies['releasedate'].dt.year

In [19]:
dfmovies.head()

,movieId,movietitle,releasedate,unknown,action,adventure,animation,childrens,comedy,crime,...,film-Noir,horror,musical,mystery,romance,sci-fi,thriller,war,western,year
0,1,Toy Story (1995),1995-01-01,0,0,0,1,1,1,0,...,0,0,0,0,0,0,0,0,0,1995.0
1,2,GoldenEye (1995),1995-01-01,0,1,1,0,0,0,0,...,0,0,0,0,0,0,1,0,0,1995.0
2,3,Four Rooms (1995),1995-01-01,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,1995.0
3,4,Get Shorty (1995),1995-01-01,0,1,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,1995.0
4,5,Copycat (1995),1995-01-01,0,0,0,0,0,0,1,...,0,0,0,0,0,0,1,0,0,1995.0


In [20]:
# encode year feature
n_dfmovies = pd.get_dummies(dfmovies, columns = ['year'])

In [21]:
# drop features in movies dataset
n_dfmovies = n_dfmovies.drop(['movieId', 'movietitle', 'releasedate'], axis=1)

In [22]:
n_dfmovies.head(), n_dfmovies.shape

(   unknown  action  adventure  animation  childrens  comedy  crime  \
 0        0       0          0          1          1       1      0   
 1        0       1          1          0          0       0      0   
 2        0       0          0          0          0       0      0   
 3        0       1          0          0          0       1      0   
 4        0       0          0          0          0       0      1   
 
    documentary  drama  fantasy  ...  year_1989.0  year_1990.0  year_1991.0  \
 0            0      0        0  ...            0            0            0   
 1            0      0        0  ...            0            0            0   
 2            0      0        0  ...            0            0            0   
 3            0      1        0  ...            0            0            0   
 4            0      1        0  ...            0            0            0   
 
    year_1992.0  year_1993.0  year_1994.0  year_1995.0  year_1996.0  \
 0            0          

In [23]:
n_dfmovies.describe()

,unknown,action,adventure,animation,childrens,comedy,crime,documentary,drama,fantasy,...,year_1989.0,year_1990.0,year_1991.0,year_1992.0,year_1993.0,year_1994.0,year_1995.0,year_1996.0,year_1997.0,year_1998.0
count,1682.000000,1682.000000,1682.000000,1682.000000,1682.000000,1682.000000,1682.000000,1682.000000,1682.000000,1682.00000,...,1682.000000,1682.000000,1682.00000,1682.000000,1682.000000,1682.000000,1682.000000,1682.000000,1682.000000,1682.000000
mean,0.001189,0.149227,0.080262,0.024970,0.072533,0.300238,0.064804,0.029727,0.431034,0.01308,...,0.008918,0.014269,0.01308,0.021998,0.074911,0.127229,0.130202,0.211058,0.170036,0.038644
std,0.034473,0.356418,0.271779,0.156081,0.259445,0.458498,0.246253,0.169882,0.495368,0.11365,...,0.094041,0.118632,0.11365,0.146719,0.263326,0.333329,0.336626,0.408181,0.375776,0.192803
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,...,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,...,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,...,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,1.000000,0.00000,...,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
max,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.00000,...,1.000000,1.000000,1.00000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [24]:
# KModes model
num_clusters = 9
kmodes = KModes(n_clusters=num_clusters, init="Huang")
kmodes.fit_predict(n_dfmovies)

array([4, 1, 1, ..., 3, 2, 5], dtype=uint16)

In [25]:
# cluster centroids
centroids = kmodes.cluster_centroids_

In [26]:
centroids.dtype, centroids.shape

(dtype('int64'), (9, 90))

In [27]:
n_dfmovies = n_dfmovies.values

In [28]:
n_dfmovies.shape

(1682, 90)

In [29]:
# compute euclidean distance between a cluster centroid and each data point
euc = EuclideanDistance(n_dfmovies, num_clusters, centroids)
eucres = euc.compute()

In [30]:
eucres.shape

(1682, 9)

In [31]:
val_m = np.array(dfmovies.movieId.values)
val_m = val_m.reshape(eucres.shape[0],1)
val_m.shape


(1682, 1)

In [32]:
# merge user feature in movies dataset
n_dfmovies = np.concatenate((val_m, n_dfmovies), axis=1)
n_dfmovies, n_dfmovies.shape

(array([[   1,    0,    0, ...,    0,    0,    0],
        [   2,    0,    1, ...,    0,    0,    0],
        [   3,    0,    0, ...,    0,    0,    0],
        ...,
        [1680,    0,    0, ...,    0,    0,    1],
        [1681,    0,    0, ...,    0,    0,    0],
        [1682,    0,    0, ...,    1,    0,    0]]),
 (1682, 91))

In [33]:
# merge movies and euclidean distances
n_dfmovies = np.concatenate((n_dfmovies, eucres), axis=1)
n_dfmovies.shape

(1682, 100)

In [34]:
n_dfmovies

array([[1.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
        2.44948974e+00, 2.44948974e+00, 2.23606798e+00],
       [2.00000000e+00, 0.00000000e+00, 1.00000000e+00, ...,
        2.44948974e+00, 2.44948974e+00, 2.23606798e+00],
       [3.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
        2.00000000e+00, 2.00000000e+00, 1.73205081e+00],
       ...,
       [1.68000000e+03, 0.00000000e+00, 0.00000000e+00, ...,
        1.73205081e+00, 1.73205081e+00, 2.00000000e+00],
       [1.68100000e+03, 0.00000000e+00, 0.00000000e+00, ...,
        2.00000000e+00, 2.00000000e+00, 1.73205081e+00],
       [1.68200000e+03, 0.00000000e+00, 0.00000000e+00, ...,
        2.00000000e+00, 1.41421356e+00, 1.00000000e+00]])

In [35]:
# import ratings dataset
dfratings = pd.read_csv(f"{DATAPATH}/u.data", sep='\t', header=None, engine='python', encoding='latin-1')
dfratings.columns = ['userId', 'movieId', 'rating', 'timestamp']

In [36]:
# transform timestamp feature
dfratings['timestamp'] = pd.to_datetime (dfratings['timestamp'], unit='s')
dfratings['year']=dfratings['timestamp'].dt.year

In [37]:
# scalerize ratings dataset
scaler = MinMaxScaler()
dfratings[['year', 'rating']]=scaler.fit_transform(dfratings[['year', 'rating']])
dfratings = dfratings.drop('timestamp', axis=1)

In [38]:
# convert dataframe to an array
n_dfratings = dfratings.drop('rating', axis=1).values
n_ratings = dfratings['rating'].values

In [39]:
# merge dataset
list =[]
for m in range(n_dfratings.shape[0]):
    for n in range(n_dfmovies.shape[0]):
        if (n_dfratings[m][1] == n_dfmovies[n][0]):
            merge = np.concatenate((n_dfmovies[n], n_dfratings[m]))
            list.append(merge)
list = np.array(list)
new_list = []
# delete duplicate movieId
for k in range(list.shape[0]):
    val = np.delete(list[k], 0)
    new_list.append(val)
new_list = np.array(new_list)

In [40]:
new_list.shape

(100000, 102)

In [41]:
# split X, y
X = new_list
y = n_ratings

In [42]:
# kfold cross validation
K = evaluation( X, y, 5, 100)
cv = K.kfold()

Fold 1:
Fold 2:
Fold 3:
Fold 4:
Fold 5:


In [43]:
#split tuple 
classes, eucdis = cv
classes, eucdis

(array([[1.42, 2.32, 2.67, ..., 3.87, 2.52, 2.81],
        [4.53, 3.53, 4.29, ..., 3.58, 2.74, 4.1 ],
        [2.45, 4.33, 3.8 , ..., 3.99, 2.39, 2.66],
        [4.14, 3.75, 3.32, ..., 3.15, 2.49, 3.85],
        [3.8 , 4.15, 4.04, ..., 2.86, 3.29, 4.17]]),
 array([[0.  , 0.5 , 0.5 , ..., 0.75, 0.5 , 0.25],
        [0.75, 1.  , 0.5 , ..., 0.25, 0.5 , 1.  ],
        [0.5 , 0.75, 1.  , ..., 0.  , 0.  , 0.  ],
        [0.5 , 0.25, 1.  , ..., 0.5 , 0.25, 0.5 ],
        [0.5 , 0.  , 0.25, ..., 0.75, 0.75, 0.5 ]]))

In [44]:
# metrics
ans = result(classes, eucdis)
Metrics = ans.validate()

Metric | Mean | Standard Deviation
RMSE 2.965250537972268, 0.0025133752563056555
MAE 2.8915148000000004, 0.0026076297398213697
